# Example: Independent Scehduler

## Imports

In [3]:
########################################################################
## FOR NOTEBOOKS ONLY: ADD THE PROJECT ROOT TO THE PYTHON PATH
########################################################################

import os
import sys

sys.path.insert(
    0, os.path.abspath(os.path.join(os.getcwd(), '..'))
)

In [4]:
import datetime

from sinkingfund.allocation.sorted import SortedAllocator
from sinkingfund.utils.loader import load_envelopes_from_csv

## Load Bills

In [5]:
path = 'data/schwab_fund.csv'
interval = 14

envelopes = load_envelopes_from_csv(path=path, contrib_intervals=interval)

## Allocate Balance

In [6]:
# The start date of the contribution schedule. The current balance must
# match the balance at the start date. If there are unpaid bills in the
# account, you manually need to remove their amounts from the balance.
start_date = datetime.date(2025, 6, 1)
balance = 15026.41

### Cascading Allocation

In [7]:
# Initialize the allocator.
allocator = SortedAllocator(sort_key='cascade')

# Allocate the balance to the envelopes.
allocator.allocate(envelopes=envelopes, balance=balance, curr_date=start_date)

# Verify total allocated equals budget.
print(sum([e.allocated for e in envelopes if e.allocated is not None]) == balance)
print([(e.bill.bill_id, e.bill.amount_due, e.allocated, e.remaining) for e in envelopes])

True
[('car_insur_1', 774.76, None, None), ('suburu_reg', 191.0, 0.0, 191.0), ('honda_reg', 334.0, 0.0, 334.0), ('prop_tax_supl_2024', 11520.23, 11520.23, 0.0), ('car_insur', 774.76, 774.76, 0.0), ('prop_tax_1', 13161.15, 2731.42, 10429.73), ('prop_tax_2', 13161.15, 0.0, 13161.15), ('home_insur', 3000.0, 0.0, 3000.0)]


## Schedule Contributions

In [8]:
# Initialize the scheduler.
scheduler = IndependentScheduler()

# Schedule the contributions.
scheduler.schedule(envelopes=envelopes, start_date=start_date)

# Verify (1) that the total contributions equal to the amount remaining;
# (2) that the total contributions plus the initial allocation equals
# the amount due.
for e in envelopes:
    
    if e.schedule is None:
        continue

    contrib = sum(cf.amount for cf in e.schedule if cf.amount > 0)
    print(e.bill.bill_id, contrib == e.remaining, contrib, e.remaining)
    print(e.bill.bill_id, contrib + e.allocated == e.bill.amount_due, contrib, e.remaining)

suburu_reg True 191.0 191.0
suburu_reg True 191.0 191.0
honda_reg True 334.0 334.0
honda_reg True 334.0 334.0
prop_tax_supl_2024 True 0 0.0
prop_tax_supl_2024 True 0 0.0
car_insur True 0 0.0
car_insur True 0 0.0
prop_tax_1 True 10429.73 10429.73
prop_tax_1 True 10429.73 10429.73
prop_tax_2 False 13161.150000000001 13161.15
prop_tax_2 False 13161.150000000001 13161.15
home_insur True 3000.0 3000.0
home_insur True 3000.0 3000.0
